<p style="text-align: center">
    <b><span style="float: center; font-size: 12pt">LocalThingsNetwork</span>
    <br>
    <span style="font-size: 40pt"> Client demonstration </span></b>
</p>

---

This is a demonstration of the `localthingsnet.network.Client` class. It is shown how to **find** and **connect** to a server, **interact** with the server through text, commands and requests as well as how to customize and **modify** the client.

---

In [1]:
import localthingsnet.network as ltn

In [2]:
# Initialise a server for demonstration purposes
# NOTE: All Server outputs except warnings are hidden to focus on the client

server = ltn.Server(servername='demoserver', description="A simple demonstration")
server.startServer(maindatasock=True)

def mute(self, *args, **kwargs):
    pass
server.printInfo = mute.__get__(server, ltn.Server)
server.connectionInfo = mute.__get__(server, ltn.Server)
server.statusInfo = mute.__get__(server, ltn.Server)
server.errorInfo = mute.__get__(server, ltn.Server)
# s.warningInfo = mute.__get__(s, Server)

Serversock bound to 4000
Server is running at ('192.168.178.140', 4000)
Maindata Datasock bound to 3999


---
---

# ***Client initialization***

The setup of a client is straight forward. The client has four optional parameters `username`, `description`, `logfile` and `ansi`:

>***username***  
> * A name to address the client. (Can be changed later on)
> * *(Default is '')*

>***description***  
> * Short description about the client.
> * *(Default is "None")*

>***logfile***  
> * Save all occurred events to this file.
> * *(Default is '', no log file created)*

>**ansi**  
> * Allow ANSI text formatting.
> * *(Default is True)*

In [3]:
client = ltn.Client(
    username='Demo_client',
    description="Simple client for a demonstration")

client.__version__

'6.26.136'

To connect the client to a server we need to know the server address. The client provides a method `searchForServer()` that searches for active servers on certain addresses or the hole local network. Found server addresses get returned and by default internally stored *(in `client.autoconnect_addrs`)* . This can be changed through the **add_to_autoconnect** parameter. If you only need a single server you can enable the **only_one** parameter which aborts the search after the first server is found.

In [4]:
addrs = client.searchServers(ports=(4000, 4001), ips='locals', only_one=False, log_info=True)
print("\nFound addresses:", client.autoconnect_addrs)

Searching for servers on 256 IPs at 2 ports.
Found Server on 192.168.178.140 at 4000

Found addresses: [('192.168.178.140', 4000)]


---
# ***Connect***

The client can be connected to a server with the `connect()` function. Connections are made either through entering an address tuple *(direct connect)* or through an 'already known' address in the `c.autoconnect_addrs` list *(autoconnect)*. The function tries to connect to an address and returns a string informing about success or failure. Once the Client connects to an active server, a registration protocol is carried out to ensure the compatibility between Server and Client. Also general information like servername, version, available servercommands and -requests, etc. are exchanged during the registration. If the client is connected with a server prior to a connect attempt, the client will disconnect from that server and connect to the new one instead.

In [5]:
# Direct connect
info = client.connect((client.ip, 4000))

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4000
Connected new datasocket to port 3999
Connected to demoserver
Registered as 'Demo_client'


Type 's.help()' for more information
You have admin permissions


In [ ]:
# Autoconnect (optional)
info = client.connect(autoconnect=True)

The client is now connected with the Server as indicated by the console outputs.

---

# ***Communication***

A good way to start interacting with the server is by sending `"s.help()"` to the server as instructed by the output above. As we will later learn this is a *servercommand* that can show a lot of useful information about the server and its abilities. Text messages can be transmitted to the server with `sendMsg()`. Other send methods will be introduced shortly.

In [6]:
client.sendMsg("s.help()")

SERVER HELP:
  If you don't know how to interact/operate this
  server then please refer to the usage examples on
  https://github.com/OmegaDawn/localthingsnet 

  Information about the server and the underlying
  project can be gained by sending 's.help(server)'
  and 's.help(project)'. For available
  servercommands type 's.help(commands)'.


In [7]:
client.sendMsg("s.help(project)")

LOCALTHINGSNETWORK PROJECT:
  This is an application of the localthingsnet(work)
  project(https://github.com/OmegaDawn/localthingsnet)
  The project aims to provide a socket based
  communication tool for personal DIY and/or IoT
  projects that need a simple and easy to setup
  connection to other devices. Further information can
  be found in the already mentioned repository.


Lets's  look at other ways the client can communicate with the server. As shown `sendMsg()` sends a text message to the server. This function uses the main socket of the server (called ***serversock***) for the transmission. The user mostly interacts with the server through the serversock. Server may hold additional ***datasocks*** that are used for background- and special data transmission. The server connects the client with a datasock if needed. The client can send data through any connected socket with `sendData()`. The data will be encoded with `pickle.dumps()` and sent to the server. Most python objects including strings, numbers, lists or class objects can be transmitted this way. All connected sockets are stored in the `client.clientsocks` dictionary, addressable through their connected port. A third way of interacting is through `sendRequest()`. This allows to request and return certain data from the server. Available *requestables* are defined by the server.

>***Client.sendMsg(msg: str)***
>- For user inputs
>- Sends a message or command through the serversock
>- Build upon `sendData()`

>***Client.sendData(sock: socket.socket, data: object)***
>- For transmitting data 
>- Binary encodes python objects and send them through the given socket
>- Useable for all connected sockets

>***Client.sendRequest(sock: socket.socket, request: str, timeout: float = 1.0) -> object***
>- Gets data from the server
>- Everything that can be requested is set server side
>- Mainly used with the maindatasock

> ***Note:** The snipped below may throw a KeyError if the server- and maindatasock are not bound to port 4000 and 3999. The ports the client is connected to can be ssen in the connect output above or the keys in the `client.clientsocks` dictionary. The serversock port can also be gained with `client.connected_addr[1]`.*

In [8]:
# Send text messages
client.sendMsg("Hello server!")
client.sendData(client.clientsocks[4000], "Sent to the serversock")  # This is equal to sendMsg()
client.sendData(client.clientsocks[3999], "Send to the maindatasock")

Received: Hello server!
Received: Send to the maindatasock
Received: Sent to the serversock


In [9]:
# Send data
client.sendData(client.clientsocks[4000], ["This is a list", 1, 2])
client.sendData(client.clientsocks[3999], False)

WAR: Could not identify the purpose of a <class 'list'> obj sent by user 'Demo_client'
WAR: Could not identify the purpose of a <class 'bool'> obj sent by user 'Demo_client'


> ***Note:** The lines above display a warning, because the server is not prepared to receive data other than strings. (Refer to the Server demonstration on how to handle different data formats with custom receiving functions (recvFunc))*

<br>

The next line requests live data from the server through the use of `sendRequest()`. In this case metadata of the server like active sockets, available servercommands, version, etc. is requested.

In [10]:
client.sendRequest(client.clientsocks[client.connected_addr[1]], 'SERVERDATA')

{'servername': 'demoserver',
 'description': 'A simple demonstration',
 'server_version': '7.43.162',
 'starttime': 1672317672.5072393,
 'layer': 0,
 'separator': '<$SEP$>',
 'max_datasize': 1024,
 'max_username_length': 16,
 'servercommands': ['s.connectto',
  's.ping',
  's.changename',
  's.getadmin',
  's.removeadmin',
  's.mute',
  's.unmute',
  's.kickuser',
  's.kickip',
  's.getrights',
  's.removerights',
  's.setadminkey',
  's.storelog',
  's.closedatasock',
  's.restart',
  's.shutdown',
  's.services',
  's.getadminkey',
  's.help',
  's.info',
  's.errortrace',
  's.attributes',
  's.listthreads',
  's.listsocks'],
 'requestables': ['SERVERDATA'],
 'socks': [['Serversock', 4000], ['Maindata', 3999]]}

---
### ***Servercommands***

Server provide a set of servercommands that allows the user to control the server. Every message starting with *'s.'* and with brackets will be interpreted as a servercommand. Servercommands can have arguments although, they must be passed as strings. The `s.help()` servercommand was already presented. One argument that wasn't used before is the *commands* option. By sending `s.help(commands)`, a list of all available commands and needed/optional arguments will be displayed. The `s.help()` command also provides the possibility to pass a command name as argument to get more information about that command. Commands require certain permissions to be executed. Every client has the *user* permission by default. Admin permissions can be gained with the `s.getadmin()` servercommand and the password as argument. Other permissions can be gained with `s.getrights()` if the user is an admin.

>***Note:** For the given demoserver every connection gets admin permissions automatically* since the adminkey was set to '' at the server initialization.

In [11]:
client.sendMsg("s.help(commands)")

AVAILABLE SERVERCOMMANDS:
  Use "s.help(*command_name*)" for more
    information about that command
  Parameters with a '[o]' are optional
  Parameters with a '[r]' are repeatable

  server management commands:
    s.closedatasock(serversockname[r])
    s.restart()
    s.services(enable, service_name[o])
    s.setadminkey(new_key)
    s.shutdown()
    s.storelog(filename[o])
  statistic commands:
    s.attributes()
    s.errortrace()
    s.getadminkey()
    s.help(servercommand[o][r])
    s.info(entity_name[o][r])
    s.listsocks()
    s.listthreads()
  user management commands:
    s.changename(newname, clientname[o])
    s.connectto(ip, port[o])
    s.getadmin(adminkey[o])
    s.getrights(permission[r])
    s.kickip(ip[r])
    s.kickuser(username[r])
    s.mute(username[r])
    s.ping()
    s.removeadmin(username[o][r])
    s.removerights(permission[r])
    s.unmute(username[r])


In [12]:
client.sendMsg("s.getrights(to_have_fun, to_dance)")

Gained permissions: to_have_fun, to_dance


In [13]:
client.sendMsg("s.removerights(to_dance)")

Removed permissions: to_dance


In [14]:
client.sendMsg("s.setadminkey(1234)")
client.sendMsg("s.getadminkey()")

Changed adminkey to '1234'
Adminkey: 1234


---
---

# ***Modifications***

Client and Server are constructed to be highly extendable and new commands, requestables, and special datasockets can be defined if needed. 

### ***Clientcommands***

Like the server, the client also has commands named **clientcommands**. They get called through the server and are used to to update client variables or execute certain actions like connecting to a datasock of the server. New clientcommands can be defined or updated with `newClientCommand()` and removed with `delClientCommand()`

In [15]:
def multiply(*args):
    print("Product:", args[0] * args[1])

client.newClientCommand(
    name="multiply",
    action=multiply,
    overwrite=False)

In [16]:
# Server side get socket of the client (see server demonstration for more)
user_sock = list(server.conns.values())[0]['socks'][server.getMainDataPort()]

# Call clientcommand through server
server.sendCommandTo(user_sock, 'multiply', (3, 7))

Product: 21


It is also possible to pass the socket that received the command and is now calling it. This is done by naming the first argument `calling_socket` as shown in the example below.

In [17]:
def show_caller_and_args(calling_socket, *args):
    print("Receiving socket:", calling_socket)
    print("Arguments:", args)

client.newClientCommand(
    name='caller_info',
    action=show_caller_and_args)

# Call the clientcommand through server
user_sock = list(server.conns.values())[0]['socks'][server.getMainDataPort()]
server.sendCommandTo(user_sock, 'caller_info', (123, 4))

Receiving socket: <socket.socket fd=1464, family=2, type=1, proto=0, laddr=('192.168.178.140', 50807), raddr=('192.168.178.140', 3999)>
Arguments: (123, 4)


In [18]:
client.delClientCommand('multiply')
client.delClientCommand('caller_info')

### ***Requestables***

(Client-)requestables allow the server to get (live) data from the client. Available requestables are set by the client. The functions to create and delete Requestables are similar to client commands. New Requestables can be defined or existing ones overwritten with `newRequestable()` and deleted with `delClientRequestable()`.

In [19]:
client.newClientRequestable(
    name='Give me some numbers',
    data=[34.5, 2, 6.08695],
    overwrite=False)

In [20]:
# server requests from the client
user_sock = list(server.conns.values())[0]['socks'][server.getMainDataPort()]
server.sendRequestTo(user_sock, 'Give me some numbers')

[34.5, 2, 6.08695]

---

# ***Disconnect***

A client can be disconnected from a server with a `disconnect()` call or by kicking the client *(f.e. with servercommand `s.kickuser(Demo_client)`)*. Connecting to a new server or terminating the server will also correctly close the connection with the server.

In [21]:
client.disconnect()

# Other ways to disconnect:
#c.sendMsg("s.kickuser(Demo_client)")
#c.connect(('192.0.0.1', 0))  # or other address

Lost connection to socket at 4000Disconnected from Server 'demoserver'

Lost connection to socket at 3999


---

# ***Outputs***

Every output of the client program is made through one of the following functions. These functions can be modified and integrated into other applications.

> ***Client.printInfo(message: str, sender: str)***
> * Outputs received server messages

> ***Client.statusInfo(status: str)***
> * Outputs system messages

> ***Client.warningInfo(warning: str)***
> * Outputs warnings

> ***Client.errorInfo(error: str)***
> * Outputs errors

> ***Client.logEvent(event: str)***
> * Stores every occurred output internally and in the logfile if one is set


---

In [22]:
# Shutdown testserver
server.shutdownServer()

---
<span style="float: right; font-size: 15pt"><b>LocalThingsNetwork</b></i></span>
<br><br>
<span style="float: right; font-size: 10pt"><i>For Client version </i><b>6.26.136</b></span>